In [1]:
__author__="Katie Kosak"
__email__="mkosak2011@my.fit.edu"
__email__="katie.kosak@gmail.com"
__credits__=["KunYang Li", "Dr. Eric Perlman", "Sayali Avachat"]
__status__="production"

from scipy.optimize import fsolve,anneal,curve_fit
from scipy import array
import pyfits
import numpy as np
from numpy import convolve
import matplotlib.pyplot as plt
import math
from pylab import *
from matplotlib import *
from scipy.interpolate import spline
from mpl_toolkits.mplot3d import Axes3D
from numpy import ma
from matplotlib import scale as mscale
from matplotlib import transforms as mtransforms
import matplotlib.ticker as ticker
s=[]
eta=[]
def imagesqrt(image, scale_min, scale_max):
        # Algorithm Courtesy of Min-Su Shin (msshin @ umich.edu) 
        # Modified by Katie Kosak 07/02/2015 to fit the needs
        # of Dr. Perlman's Research
   imageData=np.array(image, copy=True)
   imageData = imageData - scale_min
   indices = np.where(imageData < 0)
   imageData[indices] = 0.0
   imageData = np.sqrt(imageData)
   imageData = imageData / math.sqrt(scale_max - scale_min)
   return imageData

ImportError: cannot import name 'anneal'

In [2]:
def moving_average(x,y,step_size=0.1,width=4):
    bin_centers  = np.arange(np.min(x),np.max(x)-0.5*step_size,step_size)+0.5*step_size
    bin_avg = np.zeros(len(bin_centers))
    def gaussian(x,amp=1,mean=0,sigma=1):
        return amp*np.exp(-(x-mean)**2/(2*sigma**2))

    for index in range(0,len(bin_centers)):
        bin_center = bin_centers[index]
        weights = gaussian(x,mean=bin_center,sigma=width)
        bin_avg[index] = np.average(y,weights=weights)

    return (bin_centers,bin_avg)


In [ ]:
## Open the image from fits file 
## Obtain image parameters
## 4 pixels
nameofknots='m87_newknotD'
filename=nameofknots+'.fits'
file1=pyfits.getdata(filename)
pixel_min=np.nanmin(file1)
pixel_maxima=np.nanmax(file1)
file1=imagesqrt(file1,pixel_min,pixel_maxima) # Square Root Scaling for fits image
plt.imshow(file1)
## Go column by column to calculate the max flux for each column
height, width=file1.shape
intensity_max=[]
intensity_ypos=[]
intensity_xpos=[]
temp=np.zeros(shape=(height,1))
for k in range(0,width-1):
    for j in range(0,height-1):
        pixel=file1[j,k]
        temp[j]=pixel
    pixel_max=max(temp)
    position=[i for i, j in enumerate(temp) if j == pixel_max] 
    intensity_max.append(pixel_max)
    intensity_ypos.append(position[0])
    intensity_xpos.append(k)
intensity_max=np.array(intensity_max)
intensity_ypos=np.array(intensity_ypos)
intensity_xpos=np.array(intensity_xpos)

In [ ]:

## Find the average of each column of the data
intensity_average=[]
temp_pos=np.zeros(shape=(height,1))
temp_intensity=np.zeros(shape=(height,1))
for k in range(width-1):
    for j in range(height-1):
        pixel=file1[j,k]
        temp_pos[j]=j
        temp_intensity[j]=pixel
    bin_centers,pixel_average=moving_average(temp_pos,temp_intensity)  
    intensity_average.append(pixel_average)
x=intensity_xpos
y=intensity_ypos
#popt,pcov=curve_fit(gaussian,x,y)

#x_smooth = np.linspace(x.min(), x.max(), 42)
x_smooth=np.array((0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,33,34,36,38,40,42,46,48,50,52,54,56,58,60,62,64,66,67,68,70,72,74,76,78,80,82,84,86,90,92,95))
y_smooth = spline(x, y, x_smooth)

## Define Axis
#plt.contour(file1,30, cmap='gray')
plt.plot(x_smooth,y_smooth)
scale=0.025
xscale=width*scale
yscale=height*scale
plt.title(nameofknots)
ax = plt.gca()
ax.invert_yaxis()#plt.plot(xl, yl, '-r')
plt.show()

In [ ]:

## Calculate the s and eta values
for i in range(len(x_smooth)):
    s_value=(x_smooth[i]**2+y_smooth[i]**2)**(0.5)
    eta_value=math.atan2(y_smooth[i]/x_smooth[i])
    s.append(s_value)
    eta.append(eta_value)
file=open('M87_knotDparameters.txt', 'w')
for x in range(len(x_smooth)):
    file.write(str(s[x])+'\t'+ str(eta[x])+'\t'+ str(x_smooth[x]) +'\t'+ str(y_smooth[x])+'\n')
file.close()
 